<a href="https://colab.research.google.com/github/corentinBlanchard/chatbot-IA/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("a")
print("b")

a
